# Scorecard data - Further narrow feature set


This file contains work to further narrow the feature set -- got from 200 columns down to about 30 that
I played with for contstructing the linear model.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

## Read in the first pass scorecard data

In [2]:
filedir = "../data/"
filename =  filedir +  "MERGED2016_17_subset.pck" 
dataframe = pd.read_pickle(filename) 

In [3]:
dataframe.shape

(1997, 203)

## Read in the data dictionary and define a quick reference function I can use from the console

In [4]:
datad = pd.read_excel("../data/CollegeScorecardDataDictionary.xlsx", sheet_name="data_dictionary")
datad.columns = datad.columns.str.replace(" ", "_").str.replace("-", "_").str.lower()

In [5]:
# A way to quickly look up a column meaning while working on this notebook
def reference(column):
    if type(column) == str:
        ref = datad.loc[datad.variable_name == column, :]
        ref = ref.iloc[0]
        print(ref.variable_name)
        print(ref.dev_category, ref.api_data_type, ref.source)
        print(ref.name_of_data_element)
        print(ref.notes)
    else: 
        # assume it's a list or series
        for c in column:
            reference(c)
            print("")
        

# Select target column

I'm not sure what it is, but I know it came from the NSLDS (National Student Loan Data System).

Which columns in my filtered data sets came from the NSLDS? 

In [6]:
# find the columns that come from NSLDS and are in my data subset
nslds = datad.query("source == 'NSLDS'")
nslds_vars = nslds.variable_name.dropna()

In [7]:
cols = dataframe.columns

In [8]:
nslds_cols = cols[ cols.isin(nslds_vars) ]

In [9]:
nslds_cols

Index(['INC_PCT_LO', 'DEP_STAT_PCT_IND', 'DEP_INC_PCT_LO', 'IND_INC_PCT_LO',
       'PAR_ED_PCT_1STGEN', 'INC_PCT_M1', 'INC_PCT_M2', 'INC_PCT_H1',
       'INC_PCT_H2', 'DEP_INC_PCT_M1', 'DEP_INC_PCT_M2', 'DEP_INC_PCT_H1',
       'DEP_INC_PCT_H2', 'IND_INC_PCT_M1', 'IND_INC_PCT_M2', 'IND_INC_PCT_H1',
       'IND_INC_PCT_H2', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_PS',
       'APPL_SCH_PCT_GE2', 'APPL_SCH_PCT_GE3', 'APPL_SCH_PCT_GE4',
       'APPL_SCH_PCT_GE5', 'DEP_INC_AVG', 'IND_INC_AVG', 'DEBT_MDN',
       'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'LO_INC_DEBT_MDN', 'MD_INC_DEBT_MDN',
       'HI_INC_DEBT_MDN', 'DEP_DEBT_MDN', 'IND_DEBT_MDN', 'PELL_DEBT_MDN',
       'NOPELL_DEBT_MDN', 'FEMALE_DEBT_MDN', 'MALE_DEBT_MDN',
       'FIRSTGEN_DEBT_MDN', 'NOTFIRSTGEN_DEBT_MDN', 'DEBT_N', 'GRAD_DEBT_N',
       'WDRAW_DEBT_N', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N',
       'DEP_DEBT_N', 'IND_DEBT_N', 'PELL_DEBT_N', 'NOPELL_DEBT_N',
       'FEMALE_DEBT_N', 'MALE_DEBT_N', 'FIRSTGEN_DEB

In [10]:
reference("LOAN_EVER")

LOAN_EVER
aid float NSLDS
Share of students who received a federal loan while in school
Data element describes the earnings cohort (without exclusions for military and in-school deferments in the measurement year)


In [11]:
reference("DEBT_MDN")

DEBT_MDN
aid float NSLDS
The median original amount of the loan principal upon entering repayment
nan


In [12]:
reference("GRAD_DEBT_MDN")

GRAD_DEBT_MDN
aid float NSLDS
The median debt for students who have completed
nan


In [13]:
cols = ['LOAN_EVER', 'DEBT_MDN', 'GRAD_DEBT_MDN']
print(dataframe.loc[:,cols].count())

LOAN_EVER        1793
DEBT_MDN         1878
GRAD_DEBT_MDN    1835
dtype: int64


I have a few more rows for ```DEBT_MDN``` than ```GRAD_DEBT_MDN```, but I think 
```GRAD_DEBT_MDN``` is closer to my original problem statement.

In [14]:
# Drop rows where the GRAD_DEBT_MDN is not known
print(dataframe.shape)
dataframe.dropna(subset=['GRAD_DEBT_MDN'], inplace=True)
print(dataframe.shape)

(1997, 203)
(1835, 203)


## Choosing features/predictors 


In [15]:
# I start assuming all the columns will be predictors and this function
# pops columns off the list as I decide not to use them
def pop_predictor(col):
    if type(col) == str:
        try:
            predictors.remove(col)
        except ValueError:
            pass
    else:
        for c in col:
            pop_predictor(c)
        return


In [16]:
# Initial set of predictors -- based on what wasn't dropped
# in the first pass through the data

In [17]:
predictors = dataframe.columns

In [18]:
predictors

Index(['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP',
       'ACCREDAGENCY', 'INSTURL', 'NPCURL',
       ...
       'GRADS', 'ACCREDCODE', 'RET_FT4_POOLED', 'RET_PT4_POOLED',
       'RET_FT_DEN4_POOLED', 'RET_PT_DEN4_POOLED', 'POOLYRSRET_FT',
       'POOLYRSRET_PT', 'RET_FT4_POOLED_SUPP', 'RET_PT4_POOLED_SUPP'],
      dtype='object', length=203)

In [19]:
# GRAD_DEBT_MDN is my target, and the other DEBT fields are really not fair game
# e.g. debt incurred by various subsets of students would just be collinear

In [20]:
predictors = [ p for p in predictors if p.find("DEBT") == -1 ]

In [21]:
# dropping all other debt columns besides my target variable gets me down to 166 columns
len(predictors)

166

In [22]:
reference('COSTT4_A')


COSTT4_A
cost integer IPEDS
Average cost of attendance (academic year institutions)
The average annual total cost of attendance, including tuition and fees, books and supplies, and living expenses for all full-time, first-time, degree/certificate-seeking undergraduates who receive Title IV aid. It is calculated from values in the IPEDS Institutional Characteristics and Student Financial Aid components. Separate metrics are calculated for academic-year institutions and program-year institutions. For academic-year institutions, average cost of attendance represents an average of all programs and includes only full-time, first-time, degree/certificate-seeking undergraduates who first enrolled in the fall term. For non-academic-year institutions (program or continuous enrollment), average cost of attendance represents the program with the largest enrollment at the institution, and it includes full-time, first-time, degree/certificate-seeking undergraduates who first enrolled at any time du

In [23]:
costvars = ['COSTT4_A' ]
costvars += [ p for p in predictors if p.find("_PUB") != - 1]
costvars += [ p for p in predictors if p.find("_PRIV") != - 1]
dataframe[costvars].count()

COSTT4_A          1759
NPT4_PUB           653
NPT41_PUB          653
NPT42_PUB          652
NPT43_PUB          650
NPT44_PUB          645
NPT45_PUB          617
NPT4_048_PUB       653
NPT4_3075_PUB      652
NPT4_75UP_PUB      646
NUM4_PUB           653
NUM41_PUB          653
NUM42_PUB          653
NUM43_PUB          653
NUM44_PUB          653
NUM45_PUB          653
NPT4_PRIV         1107
NPT41_PRIV        1099
NPT42_PRIV        1090
NPT43_PRIV        1079
NPT44_PRIV        1034
NPT45_PRIV        1000
NPT4_048_PRIV     1104
NPT4_3075_PRIV    1096
NPT4_75UP_PRIV    1043
NUM4_PRIV         1107
NUM41_PRIV        1107
NUM42_PRIV        1107
NUM43_PRIV        1107
NUM44_PRIV        1107
NUM45_PRIV        1107
dtype: int64

In [24]:
# I'm going to keep COSTT4_A which is calculated for both public and private universities and has
# the most observations of the cost columns

In [25]:
predictors = [ p for p in predictors if p.find("_PUB") == -1 ]
predictors = [ p for p in predictors if p.find("_PRIV") == -1 ]

In [26]:
# too many testing columns -- let's filter down a bit
test_cols = [ p for p in predictors if p.find("SAT") == 0 or p.find("ACT") == 0]
test_cols

['SATVR25',
 'SATVR75',
 'SATMT25',
 'SATMT75',
 'SATVRMID',
 'SATMTMID',
 'ACTCM25',
 'ACTCM75',
 'ACTEN25',
 'ACTEN75',
 'ACTMT25',
 'ACTMT75',
 'ACTCMMID',
 'ACTENMID',
 'ACTMTMID',
 'SAT_AVG',
 'SAT_AVG_ALL']

In [27]:
dataframe.loc[:, test_cols].count()

SATVR25        1178
SATVR75        1178
SATMT25        1180
SATMT75        1180
SATVRMID       1178
SATMTMID       1180
ACTCM25        1220
ACTCM75        1220
ACTEN25        1130
ACTEN75        1130
ACTMT25        1131
ACTMT75        1131
ACTCMMID       1220
ACTENMID       1130
ACTMTMID       1131
SAT_AVG        1243
SAT_AVG_ALL    1282
dtype: int64

In [28]:
reference(['SAT_AVG', 'SAT_AVG_ALL'])

SAT_AVG
admissions float IPEDS
Average SAT equivalent score of students admitted
Shown/used on consumer website.

SAT_AVG_ALL
admissions float IPEDS
Average SAT equivalent score of students admitted for all campuses rolled up to the 6-digit OPE ID
Shown/used on consumer website.



In [29]:
# All of the testing cols are hightly correlated with each other and some further
# refinement is needed. For now, keeping the observations with the most rows
test_cols.remove('SAT_AVG')
test_cols.remove('SAT_AVG_ALL')
test_cols.remove('ACTCMMID')
pop_predictor(test_cols)

In [30]:
len(predictors)

122

In [31]:
# We still have a lot of labels and directory info which we cannot use as predictors
labels = ['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'ST_FIPS', 'REGION', 'ALIAS', 'INSTURL', 'NPCURL',
         'LATITUDE', 'LONGITUDE']

# we still need to remember to write these out for joining with collegedata.com data later
save_labels = ['INSTNM', 'CITY', 'STABBR', 'ALIAS', 'INSTURL', 'ZIP'] 

pop_predictor(labels)

In [32]:
# we have a bunch of demographics on the earnings cohort
# which doesn't really fit my problem statement since I'm 
# not looking at earnings after graduation
earnings_related = (datad.variable_name.isin(predictors)) & (datad.notes.dropna().str.find("earnings cohort") != -1)
earnings_vars = datad.loc[earnings_related, 'variable_name']
earnings_vars

1779         INC_N
1780     DEP_INC_N
1781     IND_INC_N
1782    DEP_STAT_N
1783      PAR_ED_N
1784    APPL_SCH_N
1846     LOAN_EVER
1847     PELL_EVER
1848     AGE_ENTRY
1851        FEMALE
1852       MARRIED
1853     DEPENDENT
1854       VETERAN
1855     FIRST_GEN
1856        FAMINC
1857     MD_FAMINC
1858    FAMINC_IND
Name: variable_name, dtype: object

In [33]:
pop_predictor(earnings_vars)

In [34]:
# we have a bunch of information on income levels
# which is also overkill as we cannot match a specific
# income level to a specific loan amount if the target
# variable is just the median loan amount
earnings_related = (datad.variable_name.isin(predictors)) & (datad.name_of_data_element.dropna().str.find("income") != -1)
earnings_vars = datad.loc[earnings_related, 'variable_name']
earnings_vars

1649        INC_PCT_LO
1651    IND_INC_PCT_LO
1652    DEP_INC_PCT_LO
1654        INC_PCT_M1
1655        INC_PCT_M2
1656        INC_PCT_H1
1657        INC_PCT_H2
1658    DEP_INC_PCT_M1
1659    DEP_INC_PCT_M2
1660    DEP_INC_PCT_H1
1661    DEP_INC_PCT_H2
1662    IND_INC_PCT_M1
1663    IND_INC_PCT_M2
1664    IND_INC_PCT_H1
1665    IND_INC_PCT_H2
1673       DEP_INC_AVG
1674       IND_INC_AVG
Name: variable_name, dtype: object

In [35]:
pop_predictor(earnings_vars)

In [36]:
# this column is all ones -- drop it
pop_predictor('ICLEVEL')  

In [37]:
# More stuff to drop: 
# ACCRED -- columns related accreditation
# PAR_ED_ -- columnns related to students' parents' level of education
# APPL_SCH_ -- number of schools applied to
# RET_ -- student retention rate
# POOLYRSRET_ -- more retention
drop_these = [ "ACCRED", "PAR_ED_", "APPL_SCH_", "RET_", "POOLYRSRET"]
for drop in drop_these:
    drop_cols = [ p for p in predictors if p.find(drop) == 0 ]
    pop_predictor(drop_cols)
    

In [38]:
# more to drop 
drop_these = [ 'HCM2', 'MAIN', 'NUMBRANCH',  'DEP_STAT_PCT_IND', # cash monitoring system, main/branch campus, dependency status
 'UGDS_MEN', 'UGDS_WOMEN','MENONLY','WOMENONLY', 'T4APPROVALDATE', 'D_PCTPELL_PCTFLOAN', 'UGNONDS', # men/women, school title4 approval date, non-degree seeking
 'GRADS', 'PCTFLOAN', 'LOCALE', 'OPENADMP', 'CCBASIC', # graduate students, percent of students taking loans (too collinear with loan amount)
       'UGDS_BLACK', 'UGDS_AIAN', 'UGDS_HISP'      ]  # high black enroloment is too collinear with HBCU, american indian with tribal college
                                    # hispanic enrollment and hispanic institution also correlated
pop_predictor(drop_these)

In [39]:
predictors

['HIGHDEG',
 'CONTROL',
 'CCUGPROF',
 'CCSIZSET',
 'HBCU',
 'PBI',
 'ANNHI',
 'TRIBAL',
 'AANAPII',
 'HSI',
 'NANTI',
 'RELAFFIL',
 'ADM_RATE',
 'ADM_RATE_ALL',
 'ACTCMMID',
 'SAT_AVG',
 'SAT_AVG_ALL',
 'UGDS',
 'UGDS_WHITE',
 'UGDS_ASIAN',
 'UGDS_NHPI',
 'UGDS_2MOR',
 'UGDS_NRA',
 'UGDS_UNKN',
 'PPTUG_EF',
 'COSTT4_A',
 'TUITIONFEE_IN',
 'TUITIONFEE_OUT',
 'TUITFTE',
 'INEXPFTE',
 'AVGFACSAL',
 'PFTFAC',
 'PCTPELL',
 'PFTFTUG1_EF']

In [40]:
len(predictors)

34

## checkpoint here! 

In [41]:
dataframe[predictors + save_labels + ['GRAD_DEBT_MDN']].to_pickle('checkpoint.pkl')

In [42]:
dataframe.shape

(1835, 203)

In [43]:

dataframe = pd.read_pickle('checkpoint.pkl')

In [44]:
dataframe.shape

(1835, 41)

## cleaning and checking for collinearity

In [45]:
# HIGHDEG = less than three shouldn't be there (associates, certificates, and nondegree programs)
#         = 3 offers bachelors degrees only
#         = 4 offers graduate degrees as well
# then let's make this column a more traditional 0/1 categorical flag
dataframe = dataframe.query('HIGHDEG >= 3').copy()
dataframe.shape

(1828, 41)

In [46]:
dataframe.HIGHDEG.value_counts()

4    1402
3     426
Name: HIGHDEG, dtype: int64

In [47]:
dataframe['BACH_ONLY'] = dataframe.eval('HIGHDEG == 3').astype(int)
dataframe.drop(columns='HIGHDEG', inplace=True)

In [48]:
idx = predictors.index('HIGHDEG')
predictors[idx] = 'BACH_ONLY'
dataframe.BACH_ONLY.value_counts()

0    1402
1     426
Name: BACH_ONLY, dtype: int64

In [49]:
cols = ['ADM_RATE', 'ADM_RATE_ALL']
reference(cols)

ADM_RATE
admissions float IPEDS
Admission rate
nan

ADM_RATE_ALL
admissions float IPEDS
Admission rate for all campuses rolled up to the 6-digit OPE ID
nan



In [50]:
print(dataframe[cols].count())

ADM_RATE        1525
ADM_RATE_ALL    1560
dtype: int64


In [51]:
# if the local branch admissions rate is not available,
# use the system admissions rate as a best guess
# then drop ADM_RATE_ALL as too collinear with ADM_RATE
dataframe.ADM_RATE = dataframe.ADM_RATE.fillna(value=dataframe.ADM_RATE_ALL)
dataframe.drop(columns='ADM_RATE_ALL', inplace=True)

pop_predictor('ADM_RATE_ALL')

In [52]:
# We can do a smilar thing with collinear colums
# SAT_AVG and SAT_AVG_ALL
dataframe.SAT_AVG = dataframe.SAT_AVG.fillna(value=dataframe.SAT_AVG_ALL)
dataframe.drop(columns='SAT_AVG_ALL', inplace=True)

pop_predictor('SAT_AVG_ALL')

In [53]:
cols = ['ACTCMMID', 'SAT_AVG']
dataframe.loc[:, cols].corr()

,ACTCMMID,SAT_AVG
ACTCMMID,1.000000,0.977364
SAT_AVG,0.977364,1.000000


In [54]:
# SAT and ACT scores are very collinear, both graphically and numerically

In [55]:
(dataframe.ACTCMMID.isnull() & dataframe.SAT_AVG.notnull()).sum()

57

In [56]:
(dataframe.ACTCMMID.notnull() & dataframe.SAT_AVG.isnull()).sum()

0

It looks like there is more SAT data than ACT data, so use SAT data for the predictor

In [57]:
pop_predictor('ACTCMMID')
dataframe.drop(columns='ACTCMMID', inplace=True)

In [58]:
# then let's make this column a more traditional 0/1 categorical flag
dataframe['PUBLIC'] = dataframe.eval('CONTROL == 1').astype(int)
dataframe.drop(columns='CONTROL', inplace=True)

idx = predictors.index('CONTROL')
predictors[idx] = 'PUBLIC'

In [59]:
# In state and out of state tuition are highly collinear
dataframe[['TUITIONFEE_IN', 'TUITIONFEE_OUT', 'TUITFTE', 'COSTT4_A']].corr()

,TUITIONFEE_IN,TUITIONFEE_OUT,TUITFTE,COSTT4_A
TUITIONFEE_IN,1.000000,0.905589,0.812161,0.982240
TUITIONFEE_OUT,0.905589,1.000000,0.782166,0.916137
TUITFTE,0.812161,0.782166,1.000000,0.827578
COSTT4_A,0.982240,0.916137,0.827578,1.000000


In [60]:
dataframe.TUITIONFEE_IN.count()

1763

In [61]:
dataframe.TUITIONFEE_OUT.count()

1763

In [62]:
dataframe.TUITFTE.count() 

1828

In [63]:
dataframe.COSTT4_A.count() 

1752

In [64]:
# keep the ones with the most data!
drop_these = [ 'TUITIONFEE_OUT', 'COSTT4_A' ]
dataframe = dataframe.drop(columns=drop_these)
pop_predictor(drop_these)

In [65]:
# Turn relaffil into true/false religious affiliation, as 
# there are too many churches to make each one its own predictor
dataframe['RELAFFIL'] = (dataframe.RELAFFIL > 0).astype(int)


In [66]:
# Turn carnegie categories into dummies

In [67]:
dataframe.CCUGPROF = dataframe.CCUGPROF.astype(int)
dataframe['PART_TIME'] = 'unknown'
dataframe.loc[dataframe.eval('CCUGPROF in [10,11,12,13,14,15]'), 'PART_TIME'] = 'lower'
dataframe.loc[dataframe.eval('CCUGPROF in [6,7,8,9]'), 'PART_TIME'] = 'medium'
dataframe.loc[dataframe.eval('CCUGPROF == 5'), 'PART_TIME'] = 'higher'

dataframe['TRANSFERS'] = 'unknown'
dataframe.loc[dataframe.eval('CCUGPROF in [6,8,10,12,14]'), 'TRANSFERS'] = 'lower'
dataframe.loc[dataframe.eval('CCUGPROF in [5,7,9,11,13,15]'), 'TRANSFERS'] = 'higher'

dataframe['INCLUSIVE'] = 'unknown'
dataframe.loc[dataframe.eval('CCUGPROF in [5,6,7,10,11]'), 'INCLUSIVE'] = 'inclusive'
dataframe.loc[dataframe.eval('CCUGPROF in [8,9,12,13]'), 'INCLUSIVE'] = 'selective'
dataframe.loc[dataframe.eval('CCUGPROF in [14,15]'), 'INCLUSIVE'] = 'more_selective'

dataframe.CCSIZSET = dataframe.CCSIZSET.astype(int)
dataframe['SIZECAT'] = 'unknown'
dataframe.loc[dataframe.eval('CCSIZSET in [6,7,8]'), 'SIZECAT'] =  'very_small'
dataframe.loc[dataframe.eval('CCSIZSET in [9,10,11]'), 'SIZECAT'] =  'small'
dataframe.loc[dataframe.eval('CCSIZSET in [12,13,14]'), 'SIZECAT'] =  'medium'
dataframe.loc[dataframe.eval('CCSIZSET in [15,16,17]'), 'SIZECAT'] =  'large'

dataframe['RESIDENTIAL'] = 'unknown'
dataframe.loc[dataframe.eval('CCSIZSET in [6,9,12,15]'), 'RESIDENTIAL'] =  'nonresidential'
dataframe.loc[dataframe.eval('CCSIZSET in [7,10,13,16]'), 'RESIDENTIAL'] =  'residential'
dataframe.loc[dataframe.eval('CCSIZSET in [8,11,14,17]'), 'RESIDENTIAL'] =  'highly_residential'

drop_these = ['CCUGPROF', 'CCSIZSET']
pop_predictor(drop_these)
dataframe = dataframe.drop(columns=drop_these)

# one hot encoding
cats = ['PART_TIME', 'TRANSFERS', 'INCLUSIVE', 'SIZECAT', 
        'RESIDENTIAL']
dummies = pd.get_dummies(dataframe[cats], drop_first=True)
dataframe = dataframe.drop(columns=cats)

In [68]:
# output the data to the next notebook!

In [69]:
dataframe = pd.concat([dataframe, dummies], axis=1)
dataframe.shape

(1828, 44)

In [70]:
filename =  filedir +  "MERGED2016_17_select.pck" 
dataframe.to_pickle(filename)

In [71]:
dataframe.columns

Index(['HBCU', 'PBI', 'ANNHI', 'TRIBAL', 'AANAPII', 'HSI', 'NANTI', 'RELAFFIL',
       'ADM_RATE', 'SAT_AVG', 'UGDS', 'UGDS_WHITE', 'UGDS_ASIAN', 'UGDS_NHPI',
       'UGDS_2MOR', 'UGDS_NRA', 'UGDS_UNKN', 'PPTUG_EF', 'TUITIONFEE_IN',
       'TUITFTE', 'INEXPFTE', 'AVGFACSAL', 'PFTFAC', 'PCTPELL', 'PFTFTUG1_EF',
       'INSTNM', 'CITY', 'STABBR', 'ALIAS', 'INSTURL', 'ZIP', 'GRAD_DEBT_MDN',
       'BACH_ONLY', 'PUBLIC', 'PART_TIME_lower', 'PART_TIME_medium',
       'TRANSFERS_lower', 'INCLUSIVE_more_selective', 'INCLUSIVE_selective',
       'SIZECAT_medium', 'SIZECAT_small', 'SIZECAT_very_small',
       'RESIDENTIAL_nonresidential', 'RESIDENTIAL_residential'],
      dtype='object')